## Import Library

In [1]:
# Install rdkit

def install_rdkit():
  !wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
  !chmod +x Miniconda3-latest-Linux-x86_64.sh
  !time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
  !time conda install -q -y -c conda-forge rdkit

# install_rdkit()

In [2]:
# Data

#from google.colab import files
import pandas as pd
import numpy as np
import regex as re

# rdkit
import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages/')
from rdkit import Chem
from rdkit.Chem import DataStructs

# random
import random as rd

import math

RDKit WARNING: [00:22:12] Enabling RDKit 2019.09.1 jupyter extensions


## Data

### Pre-Processing Data

In [3]:
def preprocessing_data(link_of_data) :
  dataset = pd.read_csv(link_of_data,sep=";")
  dataset = dataset.iloc[:,[0,7,9,10,11]]

  sameMol = []
  sameMolCount = {}
  dataset_handle = []

  for x in dataset.values:
    mol_name = x[0]
    if mol_name in sameMol:
      print("uye")
      x[0] = mol_name+"("+str(sameMolCount[mol_name])+")"
      sameMolCount[mol_name] += 1
    else:
      sameMol.append(x[0])
      sameMolCount[mol_name] = 1
    dataset_handle.append(x)

  #print(dataset_handle)
    
  list_of_column_dataframe = dataset.columns
  list_of_dataset_after = []

  list_of_dataset_before = np.array(dataset_handle)
  
  for data in list_of_dataset_before :
    if data[2] == "'='" :
      if data[4] == "nM" :
        if data[3] < 10000 :
          data[3] = data[3]/1000
          list_of_dataset_after.append(data)
  
  dataframe_of_dataset_after = pd.DataFrame(list_of_dataset_after)
  dataframe_of_dataset_after.columns = list_of_column_dataframe
  name_of_file_download = "Dataset_After.csv"
  dataframe_of_dataset_after.to_csv(name_of_file_download)
  #files.download(name_of_file_download)
  return dataframe_of_dataset_after




In [4]:
#preprocessing_data("https://raw.githubusercontent.com/relfarizi/Final_Project/master/Data/Dataset_Before.csv")

### Data After Pre-Processing

In [5]:
# for colab

#dataset = pd.read_csv('https://raw.githubusercontent.com/relfarizi/Final_Project/master/Data/Dataset_After.csv', index_col= 0)


dataset = pd.read_csv('Dataset_After.csv',index_col = 0)
dataset.head(5)

,Molecule ChEMBL ID,Smiles,Standard Relation,Standard Value,Standard Units
0,CHEMBL2263120,CC1(C)CC[C@]2(C(=O)OCCCCCCCCCCC(=O)O)CC[C@]3(C...,'=',0.7200,nM
1,CHEMBL2263121,CC1(C)CC[C@]2(C(=O)NC(Cc3ccccc3)C(=O)O)CC[C@]3...,'=',3.3100,nM
2,CHEMBL168,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,'=',5.0501,nM
3,CHEMBL508831,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,'=',0.5400,nM
4,CHEMBL506556,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,'=',0.6200,nM


In [6]:
#dataset = dataset.drop_duplicates(keep=False,inplace=True)
dataset = dataset.dropna()
mol_smiles = dataset.iloc[:,1].values.tolist()
row_drop = []
for i in range(len(mol_smiles)) :
  #print(mol_smiles[i])
  if len(str(mol_smiles[i])) < 5 :
    row_drop.append(i)
print(row_drop)
dataset.drop(row_drop)
dataset = dataset.iloc[:500,:]

[]


## Probability Activity

### 1. Molecule Activity
Untuk mencari jumlah dari nilai aktivitas masing2 molekul

variabel yang digunakan :
* mol_name = list nama molekul
* mol_smile = list smile molekul
* mol_act = list berisi nilai IC50
* act = nilai aktivitas tiap molekul
* sum_act = Nilai jumlah dari a

In [7]:
mol_name = dataset.iloc[:,0].values.tolist()
mol_smiles = dataset.iloc[:,1].values.tolist()
mol_act = dataset.iloc[:,3].values.tolist()
sum_act = 0
for molekul in mol_act :
  act = np.exp(-molekul)
  sum_act  += act
print(sum_act)

210.82550253137364


### 2. Probability Succes


Mencari nilai probabilitas kesuksesan untuk setiap molekul

Variabel yang digunakan :
* n = banyaknya molekul
* mol_prob = list probabilitas setiap molekul
* act = nilai aktivitas molekul
* prob = nilai probabilitas kesuksesan molekul
* sum_pro = nilai jumlah dari Probabiliras Kesuksesan




In [8]:
n = len(mol_act)
#p_avg = 1/n
mol_prob = []

# act = probabilitas kesuksesan

sum_pro = 0
for molekul in mol_act :
  #act = (molekul[1]*p_avg*n)/sum_act
  act = np.exp(-molekul)
  prob = act/sum_act
  prob = round(prob,8)
  mol_prob.append(prob)
  sum_pro += prob
print(sum_pro)


0.9999999200000004


## Cost (Trial)

Menentukan cost dari setiap molekul. cell ini baru coba di-generate hanya dengan mengalikan dengan sum_of_cost dengan probability kesuksesannya. sehingga nilai probabilitas terbesar diasumsikan memiliki cost yang besar pula

In [9]:
sum_of_cost = 500000
mol_cost = []
sum_cost = 0
for mol in mol_prob:
    cost = mol*sum_of_cost
    mol_cost.append(round(cost,4))
    sum_cost += cost
print(sum_cost)

499999.9599999998


## Fingerprint

berisi bit biner dari sifat yang menggambarkan suatu molekul (baca lagi)

Variabel yang digunakan :
* fingerprint suatu molekul (dalam bit biner)
* mol_fp = list dari fingerprint setiap molekul


In [10]:
mol_fp = []
i = 0
#print(mol_smiles[258])
for molekul in mol_smiles :
  smiles = Chem.MolFromSmiles(str(molekul))
  fp = Chem.RDKFingerprint(smiles)
  #print(i)
  i+= 1
  mol_fp.append(fp)
#print((mol_fp))

## To DataFrame

In [11]:
#df_mol = pd.DataFrame.from_dict(mol_act_dict, orient='index')
#df_mol.columns = ["IC50","Molecule Activity", "Probability Activity"]
#df_mol.sort_values(by=["IC50"], ascending=False)
#mol_name = pd.Series(mol_name)
#mol_act = pd.Series(mol_act)
#df_mol = pd.concat(mol_name,mol_act)
#print(len(mol_name))
#print(len(mol_smiles))
#print(len(mol_fp))
#print(len(mol_act))
#print(len(mol_prob))

df = pd.DataFrame({'mol_name':mol_name,
                       'mol_smiles' : mol_smiles,
                       'mol_act':mol_act,
                       'mol_fp' : mol_fp,
                       'mol_prob' : mol_prob,
                       'mol_cost' : mol_cost
                       })

df.head()

,mol_name,mol_smiles,mol_act,mol_fp,mol_prob,mol_cost
0,CHEMBL2263120,CC1(C)CC[C@]2(C(=O)OCCCCCCCCCCC(=O)O)CC[C@]3(C...,0.7200,"[0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",0.002309,1154.395
1,CHEMBL2263121,CC1(C)CC[C@]2(C(=O)NC(Cc3ccccc3)C(=O)O)CC[C@]3...,3.3100,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, ...",0.000173,86.605
2,CHEMBL168,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,5.0501,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",0.000030,15.200
3,CHEMBL508831,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,0.5400,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",0.002764,1382.065
4,CHEMBL506556,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,0.6200,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, ...",0.002552,1275.805


In [12]:
df_mol = df.iloc[:,[0,2,3,5]]
df_mol

,mol_name,mol_act,mol_fp,mol_cost
0,CHEMBL2263120,0.7200,"[0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",1154.395
1,CHEMBL2263121,3.3100,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, ...",86.605
2,CHEMBL168,5.0501,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",15.200
3,CHEMBL508831,0.5400,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",1382.065
4,CHEMBL506556,0.6200,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, ...",1275.805
...,...,...,...,...
495,CHEMBL276376(1),0.1600,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",2020.970
496,CHEMBL210965,0.0790,"[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, ...",2191.480
497,CHEMBL209304,0.1300,"[1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, ...",2082.515
498,CHEMBL211990,0.1600,"[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, ...",2020.970


## Individu
suatu fungsi yang bertujuan untuk me-return dari suatu populasi (kumpulan individu berdasarkan subset yang diberikan). 

return :
* mol_name
* rata-rata dari act
* rata-rata dari mol_fp

cara menyari diversity :

dengan menghitung semua similarity antar molekul kemudian hasilnya di rata2kan

sebagai contoh :

ada 3 (subset) molekul (A,B,C), maka menghitungnya

* A -> B
* A -> C
* B -> C

kemudian dirata2kan

In [189]:
dictDup = {}
for x in df_mol["mol_name"]:
    if x in dictDup.keys() :
        dictDup[i] += 1
    else :
        dictDup[i] = 1
print("sumDup")

for i in dictDup :
    if dictDup[i] > 1 :
        print(i,dictDup[i])

sumDup


In [226]:
n_subset = 5

def bundleIndividu(individu,dfMol = df_mol):
    #subset = np.array(subset.values.tolist())
    #print((individu))
    molName = dfMol["mol_name"].tolist()
    molAct = dfMol["mol_act"].tolist()
    molFp = dfMol["mol_fp"].tolist()
    #print(mol_name)
    #print(mol_act)
    #print(mol_fp[1])

    #cek duplicate
    sumDup = 0
    listDup = []
    dictDup = {}
    for i in individu :
        if i in dictDup.keys() :
            dictDup[i] += 1
        else :
            dictDup[i] = 1
    print("sumDup")

    for i in dictDup :
        if dictDup[i] > 1 :
            print(i,dictDup[i])

    bitString = []
    sumBit = 0
    print("ind",(individu))
    for mol in molName:
        #print("mol",mol)
        if mol in individu :
            bit = 1
            sumBit+=1
        else :
            bit = 0
        bitString.append(bit)
    print("bitstring" ,sumBit)
    fp_list = []
    n = len(molName)

    def fObjective(individu = bitString,dfMol = df_mol) :
    
        def fObj1(individu = individu,dfMol = dfMol) :
            act = []
            for idx,x in enumerate(individu) :
                if x  == 1 :
                    act.append(dfMol["mol_act"][idx])
            #print(np.mean(act))
            return np.mean(act)
        
        def fObj2(individu = individu, dfMol = dfMol) :
            fpList = []
            
            #print(n)
            molFp = []
            
            for idx,x in enumerate(individu) :
                if x == 1 :
                    molFp.append(dfMol["mol_fp"][idx])
            n = len(molFp)
            for i in range(n):
                for j in range(n):
                    if i > j :
                        tmpFp = DataStructs.FingerprintSimilarity(molFp[i],molFp[j])
                        fpList.append(tmpFp)
            #print("fplist",fpList)
            return np.mean(fpList)

        return fObj1(), fObj2()

    fObj1,fObj2 = fObjective()
    
    if [bitString,fObj1,fObj2] is not None :
        return [bitString,fObj1,fObj2]
    else :
        return []
print(bundleIndividu(population[0][0]))


sumDup
0 400
1 100
ind [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

### cek budget

mengecek apakah molekul yang terpilih (yang ada di subset) tidak melebihi budget yang diberikan

In [14]:
def is_budget_enough(subset,budget):
    subset =  subset.iloc[:,3].values
    sum_of_subset = np.sum(subset)
    if sum_of_subset <= budget :
        check = True
    else :
        check = False
    return [check,sum_of_subset]

### NSGA-II

In [15]:
def fObjective(individu,dfMol = df_mol) :
    
    def fObj1(individu = individu,dfMol = dfMol) :
        act = []
        for idx,x in enumerate(individu) :
            if x  == 1 :
                act.append(dfMol["mol_act"][idx])
        return np.mean(act)
    
    def fObj2(individu = individu, dfMol = dfMol) :
        fpList = []
        n = len(individu)
        for i in range(n):
            for j in range(n):
                if i > j :
                    tmpFp = DataStructs.FingerprintSimilarity(mol_fp[i],mol_fp[j])
                    fpList.append(tmpFp)
        return np.mean(fpList)

    return fObj1(), fObj2()
#fObjective(population[1])

## Inisialisasi populasi bit string

In [16]:
df_mol

,mol_name,mol_act,mol_fp,mol_cost
0,CHEMBL2263120,0.7200,"[0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",1154.395
1,CHEMBL2263121,3.3100,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, ...",86.605
2,CHEMBL168,5.0501,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",15.200
3,CHEMBL508831,0.5400,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",1382.065
4,CHEMBL506556,0.6200,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, ...",1275.805
...,...,...,...,...
495,CHEMBL276376(1),0.1600,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",2020.970
496,CHEMBL210965,0.0790,"[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, ...",2191.480
497,CHEMBL209304,0.1300,"[1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, ...",2082.515
498,CHEMBL211990,0.1600,"[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, ...",2020.970


In [17]:
type(df_mol["mol_name"])
nx = df_mol.shape[0]
#[rd.randint(0,len(df_mol)) for i in range(100)]

In [227]:

def initPopulation(nPortfolion,nGen,dfMol=df_mol):
    molName = dfMol["mol_name"].tolist()
    #print(len(molName))
    molAct = dfMol["mol_act"].tolist()
    molFp = dfMol["mol_fp"].tolist()
    #print(molFp)
    population = []
    
    nSubset = dfMol.shape[0]
    #print(nPortfolio)

    for i in range(nPortfolio) :
        #randIndividu = [rd.randint(0,len(df_mol)-1) for i in range(nGen)]
        #print(len(randIndividu))
        randIndividu = []
        while(len(randIndividu) < nGen) :
            gen = rd.randint(0,len(df_mol)-1)
            if gen not in randIndividu:
                randIndividu.append(gen)

        molNameList = []
        for j in randIndividu :
            molNameList.append(molName[j])
        print(len(molNameList))        
        individu = molNameList
        bInd = bundleIndividu(individu)
        #print(bInd)
        #bitString = bInd[0]
        population.append(bInd)
    return population
        

population=((initPopulation(10,100)))
print(sum(population[2][0]))

100
sumDup
ind ['CHEMBL512442(1)', 'CHEMBL312922', 'CHEMBL24141', 'CHEMBL202197', 'CHEMBL193233', 'CHEMBL24032(1)', 'CHEMBL2011707', 'CHEMBL443007', 'CHEMBL284664', 'CHEMBL108721', 'CHEMBL550291', 'CHEMBL562469', 'CHEMBL282306', 'CHEMBL506260', 'CHEMBL506934(1)', 'CHEMBL276376', 'CHEMBL312923', 'CHEMBL274435', 'CHEMBL184041', 'CHEMBL1082297', 'CHEMBL9252(1)', 'CHEMBL377989', 'CHEMBL211033', 'CHEMBL6845', 'CHEMBL1783788(1)', 'CHEMBL549505', 'CHEMBL247278', 'CHEMBL512442', 'CHEMBL57880', 'CHEMBL201909', 'CHEMBL280647', 'CHEMBL1801405', 'CHEMBL2087684', 'CHEMBL505561', 'CHEMBL264833', 'CHEMBL1778929', 'CHEMBL185726', 'CHEMBL574813', 'CHEMBL1783783(1)', 'CHEMBL237915', 'CHEMBL383437', 'CHEMBL578429', 'CHEMBL606859', 'CHEMBL1783783', 'CHEMBL246247', 'CHEMBL378614', 'CHEMBL376950', 'CHEMBL565795', 'CHEMBL428184(1)', 'CHEMBL446073', 'CHEMBL24606', 'CHEMBL48449', 'CHEMBL1823958', 'CHEMBL168', 'CHEMBL25628(1)', 'CHEMBL254301', 'CHEMBL267022', 'CHEMBL1082299', 'CHEMBL24557', 'CHEMBL286007', 'CHE

#### Generate Offspring ( crossover and mutation )

In [208]:
print((population[1][0]))

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 

In [253]:
def generateOffspring(population,dfMol = df_mol) :
    sizeOfPop = len(population)
    def generateParent(population=population, sizeOfPop = sizeOfPop):
        #sizeOfPop = len(population[0])
        parent1 = population[rd.randint(0,sizeOfPop-1)][0]
        parent2 = population[rd.randint(0,sizeOfPop-1)][0]
        #print("parent1",parent1)
        return [parent1,parent2]

    def crossover(parent,pCo = 0.5) :
        def bitToMol(parent,dfMol = dfMol) :
            print("paremt",parent)
            parent1,parent2 = parent
            molPar = []
            for par in parent :
                print("parbit",(par))
                indTmp = []
                for idx,x in enumerate(par) :
                    if x == 1 :
                        indTmp.append(dfMol["mol_name"][idx])
                print("indTmp",len(indTmp))
                molPar.append(indTmp)
            return molPar
        
        def molToBit(parent,dfMol = dfMol) :
            bitPar = []
            for par in parent :
                print("par",len(par))
                bitTmp = []
                for mol in dfMol["mol_name"] :
                    if mol in par :
                        bit = 1
                    else :
                        bit = 0
                    bitTmp.append(bit)
                bitPar.append(bitTmp)
            return bitPar



        randCo = rd.random()
        #print(randCo)
        parents = generateParent()
        parent1,parent2 = bitToMol(parents)
        #print("parent",parent1)
        #parent1.sort(reverse=True)
        #parent2.sort(reverse=True)
        #print(parent1)
        nGen = len(parent1)
        print("nGen",nGen)
        
        nPoint = 0.5
        #barier =(math.floor(len(population[0])*nPoint))
        barier = math.floor(nGen*nPoint) 

        if randCo > pCo :
            offspring1 = parent1[:barier]+parent2[barier:]
            offspring2 = parent2[:barier]+parent1[barier:]
        else :
            offspring1 = parent1[:barier]+parent2[barier:]
            offspring2 = parent2[:barier]+parent1[barier:]
        #print("oofspring",offspring1)
        #print("moltobit",sum(molToBit([offspring1,offspring2])[1]))
        return molToBit([offspring1,offspring2])

   # Flip mutation 
    def mutation(offSpring,pMut = 0.2) :
        randMut = rd.uniform(0,1)
        #print(randMut)
        newOffspring = []
        if randMut > pMut :
            for bit in offSpring :
                if bit == 1 :
                    newBit = 1
                else :
                    newBit = 0
                newOffspring.append(newBit)
            offSpring = newOffspring
        return offSpring



    
    # Start Generate
    solution = []
    while(len(solution) < 2*sizeOfPop):
        parents = generateParent()
        #print(parents)
        offSpring = crossover(parents)
        for x in offSpring :
            #print("x",x)
            solution.append(bundleIndividu(x,dfMol))
    return solution

print((generateOffspring(population)))


paremt [[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0

In [244]:
def fast_non_dominated_sort(values1, values2):
    S=[[] for i in range(0,len(values1))]
    front = [[]]
    n=[0 for i in range(0,len(values1))]
    rank = [0 for i in range(0, len(values1))]

    for p in range(0,len(values1)):
        S[p]=[]
        n[p]=0
        for q in range(0, len(values1)):
            if (values1[p] > values1[q] and values2[p] > values2[q]) or (values1[p] >= values1[q] and values2[p] > values2[q]) or (values1[p] > values1[q] and values2[p] >= values2[q]):
                if q not in S[p]:
                    S[p].append(q)
            elif (values1[q] > values1[p] and values2[q] > values2[p]) or (values1[q] >= values1[p] and values2[q] > values2[p]) or (values1[q] > values1[p] and values2[q] >= values2[p]):
                n[p] = n[p] + 1
        if n[p]==0:
            rank[p] = 0
            if p not in front[0]:
                front[0].append(p)

    i = 0
    while(front[i] != []):
        Q=[]
        for p in front[i]:
            for q in S[p]:
                n[q] =n[q] - 1
                if( n[q]==0):
                    rank[q]=i+1
                    if q not in Q:
                        Q.append(q)
        i = i+1
        front.append(Q)

    del front[len(front)-1]
    return front



In [245]:
def unpackInd(population) :
    fMaxObj1 = []
    fMaxObj2 = []
    solution = []
    for idx,individu in enumerate(population) :
        #print(idx)
        solution.append(individu[0])
        fMaxObj1.append(individu[1]) # Probabilitas Kesuksesan
        fMaxObj2.append(individu[2]) # Diversity   
    return [solution,fMaxObj1,fMaxObj2]

In [246]:
#fast_non_dominated_sort(fMaxObj1,fMaxObj2)

In [247]:
import math
def sort_by_values(list1, values):
    sorted_list = []
    while(len(sorted_list)!=len(list1)):
        if index_of(min(values),values) in list1:
            sorted_list.append(index_of(min(values),values))
        values[index_of(min(values),values)] = math.inf
    return sorted_list


In [248]:
def crowding_distance(values1, values2, front):
    distance = [0 for i in range(0,len(front))]
    #print(distance)
    #print(values2)
    sorted1 = sort_by_values(front, values1[:])
    #print("sorted1",sorted1)
    sorted2 = sort_by_values(front, values2[:])
    
    distance[0] = 4444444444444444
    distance[len(front) - 1] = 4444444444444444
    for k in range(1,len(front)-1):
        #print("k=",k)
        distance[k] = distance[k]+ (values1[sorted1[k+1]] - values2[sorted1[k-1]])/(max(values1)-min(values1))
    for k in range(1,len(front)-1):
        distance[k] = distance[k]+ (values1[sorted2[k+1]] - values2[sorted2[k-1]])/(max(values2)-min(values2))
    return distance

In [249]:
def index_of(a,list):
    for i in range(0,len(list)):
        if list[i] == a:
            return i
    return -1

### Coba NSGA

In [255]:
maxGen = 2
gen_0 = 0
nGen = 100
nPortfolio = 5 # banyaknya individu dalam satu populasi

population = initPopulation(nPortfolio,nGen)

for gen in range(maxGen) :
    bitPopulation = unpackInd(population)[0]
    fObj1 = unpackInd(population)[1]
    fObj2 = unpackInd(population)[2]
    print(fObj2)
    #nonDominatedSorted = fast_non_dominated_sort(fObj1,fObj2)

    # generate Offspring

    offsprings = generateOffspring(population)
    
    # Non Dominating

    fObj1 = unpackInd(population)[1]
    fObj2 = unpackInd(population)[2]
    #print(unpackInd(population))
    #print(population[0])
    nonDominatedSorted = fast_non_dominated_sort(fObj1,fObj2)

    # crowding distance
    #print(nonDominatedSorted)
    crowdingDistance = []
    for i in range(len(nonDominatedSorted)) :
        #print(i)
        crowdingDistance.append(crowding_distance(fObj1,fObj2,nonDominatedSorted[i]))

    newNonDominatedSorted = []
    new_solution = []
    for i in range(0,len(nonDominatedSorted)):
        for j in range(0,len(nonDominatedSorted[i])) :
            newNonDominatedSorted.append(index_of(nonDominatedSorted[i][j],nonDominatedSorted[i]))
    
        front22 = sort_by_values(newNonDominatedSorted[:], crowdingDistance[i][:])
        front = [nonDominatedSorted[i][front22[k]] for k in range(0,len(nonDominatedSorted[i]))]

        for value in front:
            new_solution.append(value)
            
            if(len(new_solution)==pop_size):
                break
                
        if (len(new_solution) == pop_size):
            break

    population = [offsprings[i] for i in new_solution]
    #print(population)

    

100
sumDup
ind ['CHEMBL197929', 'CHEMBL24003', 'CHEMBL510460', 'CHEMBL1783788', 'CHEMBL202219', 'CHEMBL510225', 'CHEMBL445301', 'CHEMBL190535', 'CHEMBL1241658', 'CHEMBL292556', 'CHEMBL449613', 'CHEMBL489670', 'CHEMBL285308', 'CHEMBL282855', 'CHEMBL1242826', 'CHEMBL462163', 'CHEMBL267143', 'CHEMBL1241315', 'CHEMBL267954', 'CHEMBL363977', 'CHEMBL211076', 'CHEMBL2113225', 'CHEMBL301909', 'CHEMBL2263111', 'CHEMBL1086229', 'CHEMBL6518', 'CHEMBL382311(1)', 'CHEMBL175241', 'CHEMBL1783774', 'CHEMBL210965', 'CHEMBL453868', 'CHEMBL253475', 'CHEMBL1778899', 'CHEMBL9252', 'CHEMBL590103', 'CHEMBL443663', 'CHEMBL1783790(1)', 'CHEMBL3098941', 'CHEMBL25558', 'CHEMBL52347', 'CHEMBL502586', 'CHEMBL24501', 'CHEMBL24606', 'CHEMBL376950', 'CHEMBL190801', 'CHEMBL267245', 'CHEMBL556913', 'CHEMBL1783786', 'CHEMBL428184(1)', 'CHEMBL140954', 'CHEMBL58032', 'CHEMBL108721', 'CHEMBL201161', 'CHEMBL1240790', 'CHEMBL416145', 'CHEMBL446073', 'CHEMBL187343', 'CHEMBL340653', 'CHEMBL3350948', 'CHEMBL1765349', 'CHEMBL108

KeyboardInterrupt: 

In [ ]:
f1 = [i * -1 for i in function1_values]
f2 = [j * -1 for j in function2_values]
plt.xlabel('Function 1', fontsize=15)
plt.ylabel('Function 2', fontsize=15)
plt.scatter(f1, f2)
plt.show()

In [252]:
# Inisialisasi Individu

def initialization_individu() :
    for i in range(sum_of_subset):
        random = rd.randint(0,len(df_mol))
        while random in subset_row:
            random = rd.randint(0,len(df_mol))
        subset_row.append(random)
#subset_row = rd.sample(df_mol.values,10)
df
list_subset = []
for i in (df_mol).index :
    if i in subset_row:
        is_subset = 1
    else :
        is_subset = 0
    list_subset.append(is_subset)
print(list_subset)

NameError: name 'subset_row' is not defined

In [47]:

#Main program starts here
pop_size = 7 # ini banyaknya individu dalam 1 populasi (subset kata pak isman mah)
max_gen = 100
nPortfolio = 6
#Initialization
min_x=-55
max_x=55

# Ini kayanya buat bikin solusi individu (0 atau 1)
#solution=[round(min_x+(max_x-min_x)*random.random(),3) for i in range(0,pop_size)] # Portofolio 
#print("sol",solution) # --> [-51.492, -31.654, -47.084, -11.823, 41.043]
population = initPopulation(nPortfolio)
gen_no=0


while(gen_no<max_gen):
    function1_values = unpackInd(population)[1] # Kumpulan function 1 dari 1 portofolio
    function2_values = unpackInd(population)[2]# Kumpulan function 2 dari 1 portofolio
    print("f1v",function1_values) # --> [-2651.4260639999998, -1001.975716, -2216.903056, -139.783329, -1684.5278489999998]
    print("f2v",function2_values) # -->  [-2861.3940639999996, -1132.5917159999997, -2409.2390560000003, -191.075329, -1524.355849]
    
    non_dominated_sorted_solution = fast_non_dominated_sort(function1_values[:],function2_values[:])
    print("non-dominated",non_dominated_sorted_solution)
    print("The best front for Generation number ",gen_no, " is")
    """  
    for valuez in non_dominated_sorted_solution[0]:
        print("valuez",valuez,solution[valuez],end=" ")
    print("\n")

    crowding_distance_values=[]
    
    for i in range(0,len(non_dominated_sorted_solution)):
        crowding_distance_values.append(crowding_distance(function1_values[:],function2_values[:],non_dominated_sorted_solution[i][:]))
    """
    solution2 = solution[:]
    
    
    
    # Generating offsprings

    offsprings = generateOffspring(population)
    
    function1_values2 = unpackInd(offsprings)[1]
    function2_values2 = unpackInd(offsprings)[2]
    non_dominated_sorted_solution2 = fast_non_dominated_sort(function1_values2[:],function2_values2[:])
    print("non-dominated sorteed =",non_dominated_sorted_solution2)
    """
        Hasil dari non_domiated_sorted
        
        From :
        [[0, 5], [1], [2], [3, 4], [6], [9], [7], [8]]

        to :

        [[0, 1, 2, 3, 4], [6], [9], [5], [7], [8]]
    
    """
    crowding_distance_values2=[]
    
    
    
    for i in range(0,len(non_dominated_sorted_solution2)):
        crowding_distance_values2.append(crowding_distance(function1_values2[:],function2_values2[:],non_dominated_sorted_solution2[i][:]))
    new_solution= []
    
    
    for i in range(0,len(non_dominated_sorted_solution2)):
        non_dominated_sorted_solution2_1 = [index_of(non_dominated_sorted_solution2[i][j],non_dominated_sorted_solution2[i] ) for j in range(0,len(non_dominated_sorted_solution2[i]))]
        
        front22 = sort_by_values(non_dominated_sorted_solution2_1[:], crowding_distance_values2[i][:])
        front = [non_dominated_sorted_solution2[i][front22[j]] for j in range(0,len(non_dominated_sorted_solution2[i]))]
        front.reverse()
        
        print("crowding_distance beres")
        for value in front:
            new_solution.append(value)
            
            if(len(new_solution)==pop_size):
                break
                
        if (len(new_solution) == pop_size):
            break

        print("value beres")

            
    # update solution
    #solution = [solution2[i] for i in new_solution]
    gen_no = gen_no + 1

f1v [2.200548793650793, 2.0391330745341616, 2.2205305901639347, 2.2049425625000003, 2.280626134969325, 2.1847235937500002]
f2v [0.3329861654302033, 0.33219927744732936, 0.3293527359236178, 0.3339066128315697, 0.3315270130326453, 0.3331756712179298]
non-dominated [[3, 4], [0, 5, 2], [1]]
The best front for Generation number  0  is


NameError: name 'solution' is not defined

In [ ]:
import matplotlib.pyplot as plt
f1 = [i * -1 for i in function1_values]
f2 = [j * -1 for j in function2_values]
plt.xlabel('Function 1', fontsize=15)
plt.ylabel('Function 2', fontsize=15)
plt.scatter(f1, f2)
plt.show()

In [ ]:
def nonDominatedsorted(p,q) :
    
    budget = 5000
    n_subset = 5

    # Fungsi buat Dominance test

    def isDominated(p,q): # success !!

        fObj1 = {"p": p[1], "q" : q[1]}
        fObj2 = {"p": p[2], "q" : q[2]}
        if fObj1["p"] > fObj1["q"] or fObj2["p"] > fObj2["q"] :
            return True
        else :
            return False
    """
    if isDominated(p,q):
        return [[p[1],q[1]],[p[2],q[2]]]
    else :
        return "Not uye"
    """

    # Fungsi buat Inisialisasi individu (Masih Random)

    def initialization_individu(df_mol,budget,n_subset) :
        i = 0
        iteration = 100
        calon_individu = df_mol.sample(n_subset)

        while(not is_budget_enough(calon_individu,budget)[0]) and i < iteration :
            calon_individu = df_mol.sample(n_subset)
            i+=1
        if is_budget_enough(calon_individu,budget)[0]:
            return individu(calon_individu)
        else :
            return []
    
    """ --- Start Non Dominated Sorted --- """

    Sp = []
    np = 0
    
    # Inisialisasi q

    q = initialization_individu(df_mol,budget,n_subset)

nonDominatedsorted(ind,ind2) 



### Try Pymoo

In [ ]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_problem, get_sampling, get_crossover, get_mutation
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

problem = get_problem("zdt5")

print(problem)

algorithm = NSGA2(pop_size=100,
                  sampling=get_sampling("bin_random"),
                  crossover=get_crossover("bin_two_point"),
                  mutation=get_mutation("bin_bitflip"),
                  eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               ('n_gen', 500),
               seed=1,
               verbose=True)

print(res)


In [ ]:
Scatter().add(res.F).show()

## For git (Can't be use yet)

file > save a copy in Github

In [ ]:
#!git init

#!git config --global user.name “relfarizi”

#!git remote add origin https://github.com/relfarizi/Final_Project.git

#!git add .

#!git commit -m "Probability Activity"

#!git push

#!git status

### lampau

In [ ]:
subset_row = []
sum_of_subset = 10

for i in range(sum_of_subset):
    random = rd.randint(0,len(df_mol))
    while random in subset_row:
        random = rd.randint(0,len(df_mol))
    subset_row.append(random)
    
print(subset_row)
calon_individu = df_mol.iloc[subset_row,:].values.tolist()
calon_individu

individu(calon_individu)